# Covid19 Timeline
## Using data from [Johns Hopkins CSSE](https://gisanddata.maps.arcgis.com/apps/opsdashboard/index.html#/bda7594740fd40299423467b48e9ecf6)

In [81]:
import pandas as pd
import folium
from folium.plugins import HeatMap
from folium.plugins import HeatMapWithTime
from datetime import datetime
import os


Recover Data from Github repository

In [85]:
import requests
from os import getcwd

url = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv"
filename = os.path.join(getcwd(),'covid19.csv')

r = requests.get(url)

f = open(filename,'wb')
f.write(r.content)

28373

Examine data

In [86]:
df = pd.read_csv(r'covid19.csv')
df.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,2/26/20,2/27/20,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20
0,Anhui,Mainland China,31.8257,117.2264,1,9,15,39,60,70,...,989,989,990,990,990,990,990,990,990,990.0
1,Beijing,Mainland China,40.1824,116.4142,14,22,36,41,68,80,...,400,410,410,411,413,414,414,418,418,422.0
2,Chongqing,Mainland China,30.0572,107.8740,6,9,27,57,75,110,...,576,576,576,576,576,576,576,576,576,576.0
3,Fujian,Mainland China,26.0789,117.9874,1,5,10,18,35,59,...,294,296,296,296,296,296,296,296,296,296.0
4,Gansu,Mainland China,36.0611,103.8343,0,2,2,4,7,14,...,91,91,91,91,91,91,91,91,102,119.0


Prepare data

In [87]:
indexes = df.columns[4:-1].array
#indexes

In [88]:
columns= df.columns[:4].array
#columns

In [95]:
out = pd.DataFrame(columns=columns)
out.insert(loc=0, column='Date', value='')
out.insert(loc=5, column='Value', value='')

out.head()

,Date,Province/State,Country/Region,Lat,Long,Value


In [96]:
for index, row in df.iterrows():
#     print(row['Province/State'])
    ProvinceState = row['Province/State']
    CountryRegion = row['Country/Region']
    Lat =  row['Lat']
    Long = row['Long']
    for index in indexes:
        Date = datetime.strptime(index, '%m/%d/%y').strftime("%Y-%m-%d")
        Value = row[index]
        new_row =pd.Series( data = {'Date':Date, 'Province/State':ProvinceState, 'Country/Region':CountryRegion, 'Lat':Lat, 'Long':Long, 'Value':Value}, name='record')
        out = out.append(new_row, ignore_index=True )
        
out.head()

,Date,Province/State,Country/Region,Lat,Long,Value
0,2020-01-22,Anhui,Mainland China,31.8257,117.2264,1
1,2020-01-23,Anhui,Mainland China,31.8257,117.2264,9
2,2020-01-24,Anhui,Mainland China,31.8257,117.2264,15
3,2020-01-25,Anhui,Mainland China,31.8257,117.2264,39
4,2020-01-26,Anhui,Mainland China,31.8257,117.2264,60


Write a CSV file suitable for QGIS Timeline plugin

In [97]:
# Prepare output for QGIS
out.to_csv(r'covid19_pivot.csv', index=False, date_format='%Y-%m-%d')

Prepare Maps

In [98]:
def generateBaseMap(default_location=[47.521671, 8.373731], default_zoom_start=5):
    base_map = folium.Map(location=default_location, control_scale=True, zoom_start=default_zoom_start)
    return base_map

Prepare data for map visualization

In [108]:
df_copy = out.copy()
# df_copy['count'] = 1
# base_map = generateBaseMap()
# HeatMap(data=df_copy[['Lat', 'Long', 'Value']].groupby(['Lat', 'Long']).sum().reset_index().values.tolist(), radius=8, max_zoom=13).add_to(base_map)
# base_map

In [117]:
df_values_list = []
df_timestamps = []
dates = df_copy.Date.sort_values().unique()
for date in dates:
    df_values_list.append(df_copy.loc[df_copy.Date == date, ['Lat', 'Long', 'Value']].groupby(['Lat', 'Long']).sum().reset_index().values.tolist())
    df_timestamps.append(datetime.strftime(datetime.strptime(date, "%Y-%m-%d"), "%d-%m-%Y"))
#df_Date_list

In [118]:
base_map = generateBaseMap(default_zoom_start=5)
HeatMapWithTime(df_values_list, 
                index=df_timestamps,
                radius=3, 
                scale_radius=True, 
                gradient={0.2: 'blue', 0.4: 'lime', 0.6: 'orange', 1: 'red'}, 
                min_opacity=0.3, 
                max_opacity=0.5, 
                use_local_extrema=True,
                auto_play = True
               ).add_to(base_map)
base_map

In [121]:
#cumulated
df_values_list = []
df_timestamps = []
dates = df_copy.Date.sort_values().unique()
for date in dates:
    df_values_list.append(df_copy.loc[df_copy.Date <= date, ['Lat', 'Long', 'Value']].groupby(['Lat', 'Long']).sum().reset_index().values.tolist())
    df_timestamps.append(datetime.strftime(datetime.strptime(date, "%Y-%m-%d"), "%d-%m-%Y"))
#df_values_list[0]

In [126]:
base_map = generateBaseMap(default_zoom_start=5)
HeatMapWithTime(df_values_list, 
                index=df_timestamps,
                radius=8, 
                scale_radius=True, 
                gradient={0.2: 'blue', 0.4: 'lime', 0.6: 'orange', 1: 'red'}, 
                min_opacity=0.3, 
                max_opacity=0.5, 
                use_local_extrema=True,
                auto_play = True
               ).add_to(base_map)
base_map